In [1]:
import requests
import pandas as pd

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split

In [2]:
dtr_model_1 = DecisionTreeRegressor(random_state=1)

forest_model_1 = RandomForestRegressor(n_estimators=50,criterion="absolute_error",random_state=1)
forest_model_2 = RandomForestRegressor(n_estimators=100,random_state=1)
forest_model_3 = RandomForestRegressor(n_estimators=100,criterion="friedman_mse",random_state=1)
forest_model_4 = RandomForestRegressor(n_estimators=100,criterion="absolute_error",random_state=1)
forest_model_5 = RandomForestRegressor(n_estimators=200,criterion="absolute_error",random_state=1)

forest_models = [forest_model_1,forest_model_2,forest_model_3,forest_model_4,forest_model_5]

In [3]:
my_rating_df = pd.read_csv('ratings.csv') # Lista com todos os meus filmes e minhas notas
my_rating_df.head()

,Date,Name,Year,Letterboxd URI,Rating
0,2021-11-29,Violet,2021,https://boxd.it/oMCE,4.0
1,2021-11-29,Hot Fuzz,2007,https://boxd.it/2416,3.5
2,2021-11-29,The World's End,2013,https://boxd.it/3EiO,3.0
3,2021-11-29,Scott Pilgrim vs. the World,2010,https://boxd.it/1Aq6,4.0
4,2021-11-29,La La Land,2016,https://boxd.it/a5fa,3.0


In [4]:
# Minhas configurações e API KEYs
import json

with open("config.json","r") as file:
    config = json.load(file)

In [5]:
from rapidfuzz import process, fuzz
from unidecode import unidecode
import time


# CONFIGURAÇÕES INICIAIS

test_df = my_rating_df.head(10)  # DataFrame de teste

API_KEY = config['TMDB_API_KEY']  
session = requests.Session()  
tmdb_rating = []  # Lista para armazenar resultados


# PRÉ-AQUECIMENTO DA CONEXÃO (COLD START FIX)

try:
    session.get(
        "https://api.themoviedb.org/3/search/movie",
        params={
            'api_key': API_KEY,
            'query': "dummy",
            'year': 2023
        },
        timeout=10
    )
except Exception as e:
    print(f"Aquecimento falhou (não crítico): {str(e)}", flush=True)

In [6]:

# SOLICITANDO DETALHES DE TODOS OS MEUS FILMES NA API DO TMDB

start_total = time.time()

for row in my_rating_df.itertuples():
    filme = row.Name
    ano_original = int(row.Year)
    print(f"\n--- Processando: {filme} ({ano_original}) ---", flush=True)
    
    try:
        # ETAPA 1: BUSCA NA API
        print(f"Enviando requisição...", flush=True)
        response = session.get(
            "https://api.themoviedb.org/3/search/movie",
            params={
                'api_key': API_KEY,
                'query': unidecode(filme).lower(),
                'year': ano_original,
                'page': 1
            },
            timeout=15
        )
        
        if response.status_code != 200:
            print(f"Erro na API: {response.status_code}", flush=True)
            continue
            
        data = response.json()
        results = data.get('results', [])[:5]  # Limita a 5 resultados
        
        # ETAPA 2: SELEÇÃO DO MELHOR MATCH
        best_match = None
        best_score = 0
        best_year_diff = float('inf')
        
        for result in results:
            # Compara títulos
            title_tmdb = unidecode(result.get('title', '')).lower()
            original_title = unidecode(result.get('original_title', title_tmdb)).lower()
            
            # Calcula similaridade
            score = max(
                fuzz.ratio(unidecode(filme).lower(), title_tmdb),
                fuzz.ratio(unidecode(filme).lower(), original_title)
            )
            
            # Verifica ano
            release_date = result.get('release_date', '')
            try:
                release_year = int(release_date[:4]) if release_date else None
            except ValueError:
                release_year = None
            
            year_diff = abs(release_year - ano_original) if release_year else 9999
            
            # Critérios de prioridade
            if score > 85 and year_diff <= 2:
                if (score > best_score) or \
                   (score == best_score and year_diff < best_year_diff):
                    
                    best_match = result
                    best_score = score
                    best_year_diff = year_diff
        
        if not best_match:
            print(f"Nenhum match válido", flush=True)
            continue
            
        # ETAPA 3: VERIFICAÇÃO DE DUPLICATAS
        movie_id = best_match['id']
        print(f"Buscando detalhes (ID: {movie_id})...", flush=True)
        # Verifica se já foi adicionado (por ID)
        duplicate = any(
            (item['tmdb_id'] == movie_id) 
            for item in tmdb_rating
        )
        
        if duplicate:
            for item in tmdb_rating:
                print(item)
            print(f"Duplicata evitada: {filme} ({ano_original})", flush=True)
            continue
            
        # ETAPA 4: BUSCA DE DETALHES
        response_details = session.get(
            f"https://api.themoviedb.org/3/movie/{movie_id}",
            params={'api_key': API_KEY},
            timeout=15
        )
        
        if response_details.status_code != 200:
            print(f"Erro nos detalhes: {response_details.status_code}", flush=True)
            continue
            
        details = response_details.json()
        
        # ETAPA 5: ARMAZENAMENTO DOS DADOS
        tmdb_rating.append({
            'title': filme,
            'tmdb_id': movie_id,
            'tmdb_title': details.get('title'),
            'year': details.get('release_date')[:4] if details.get('release_date') else None,
            'genre_ids': [genre['id'] for genre in details.get('genres',[])],
            'original_language': details.get('original_language'),
            'popularity': details.get('popularity'),
            'vote_count': details.get('vote_count'),
            'vote_average': details.get('vote_average'),
            'budget': details.get('budget'),
            'revenue': details.get('revenue'),
            'runtime': details.get('runtime'),
            'production_company_id': details.get('production_companies', [{}])[0].get('id') if details.get('production_companies') else None
        })
        print(f"Adicionado: {details.get('title')} ({movie_id}) | Tempo: {time.time() - start_total:.2f}s")
        
    except Exception as e:
        print(f"Erro: {str(e)}", flush=True)


# PÓS-PROCESSAMENTO

tmdb_rating_df = pd.DataFrame(tmdb_rating)

# Verificação final
print(f"\nTotal original: {len(my_rating_df)}")
print(f"Total encontrado: {len(tmdb_rating_df)}")

# Remove duplicatas residuais por ID
tmdb_rating_df = tmdb_rating_df.drop_duplicates(subset=['tmdb_id'])

print(f"\nResultado final: {len(tmdb_rating_df)} registros")
print(f"Tempo total: {time.time() - start_total:.2f}s")


--- Processando: Violet (2021) ---
Enviando requisição...
Buscando detalhes (ID: 665901)...
Adicionado: Violet (665901) | Tempo: 0.24s

--- Processando: Hot Fuzz (2007) ---
Enviando requisição...
Buscando detalhes (ID: 4638)...
Adicionado: Hot Fuzz (4638) | Tempo: 0.48s

--- Processando: The World's End (2013) ---
Enviando requisição...
Buscando detalhes (ID: 107985)...
Adicionado: The World's End (107985) | Tempo: 0.75s

--- Processando: Scott Pilgrim vs. the World (2010) ---
Enviando requisição...
Buscando detalhes (ID: 22538)...
Adicionado: Scott Pilgrim vs. the World (22538) | Tempo: 0.98s

--- Processando: La La Land (2016) ---
Enviando requisição...
Buscando detalhes (ID: 313369)...
Adicionado: La La Land (313369) | Tempo: 1.22s

--- Processando: Attack the Block (2011) ---
Enviando requisição...
Buscando detalhes (ID: 59678)...
Adicionado: Attack the Block (59678) | Tempo: 1.46s

--- Processando: Dune (2021) ---
Enviando requisição...
Buscando detalhes (ID: 438631)...
Adicionad

Buscando detalhes (ID: 1375)...
Adicionado: Rocky V (1375) | Tempo: 13.67s

--- Processando: Rocky II (1979) ---
Enviando requisição...
Buscando detalhes (ID: 1367)...
Adicionado: Rocky II (1367) | Tempo: 13.90s

--- Processando: Rocky III (1982) ---
Enviando requisição...
Buscando detalhes (ID: 1371)...
Adicionado: Rocky III (1371) | Tempo: 14.13s

--- Processando: Rocky IV (1985) ---
Enviando requisição...
Buscando detalhes (ID: 1374)...
Adicionado: Rocky IV (1374) | Tempo: 14.36s

--- Processando: Rocky Balboa (2006) ---
Enviando requisição...
Buscando detalhes (ID: 1246)...
Adicionado: Rocky Balboa (1246) | Tempo: 14.60s

--- Processando: The Expendables (2010) ---
Enviando requisição...
Buscando detalhes (ID: 27578)...
Adicionado: The Expendables (27578) | Tempo: 14.84s

--- Processando: Creed (2015) ---
Enviando requisição...
Buscando detalhes (ID: 312221)...
Adicionado: Creed (312221) | Tempo: 15.07s

--- Processando: Bruce Almighty (2003) ---
Enviando requisição...
Buscando det

Enviando requisição...
Buscando detalhes (ID: 269149)...
Adicionado: Zootopia (269149) | Tempo: 26.20s

--- Processando: Big Hero 6 (2014) ---
Enviando requisição...
Buscando detalhes (ID: 177572)...
Adicionado: Big Hero 6 (177572) | Tempo: 26.43s

--- Processando: Frozen (2013) ---
Enviando requisição...
Buscando detalhes (ID: 109445)...
Adicionado: Frozen (109445) | Tempo: 26.67s

--- Processando: Silver Linings Playbook (2012) ---
Enviando requisição...
Buscando detalhes (ID: 82693)...
Adicionado: Silver Linings Playbook (82693) | Tempo: 26.90s

--- Processando: The Nice Guys (2016) ---
Enviando requisição...
Buscando detalhes (ID: 290250)...
Adicionado: The Nice Guys (290250) | Tempo: 27.14s

--- Processando: V for Vendetta (2005) ---
Enviando requisição...
Buscando detalhes (ID: 752)...
Adicionado: V for Vendetta (752) | Tempo: 27.37s

--- Processando: Moneyball (2011) ---
Enviando requisição...
Buscando detalhes (ID: 60308)...
Adicionado: Moneyball (60308) | Tempo: 27.60s

--- Pr

Enviando requisição...
Buscando detalhes (ID: 9654)...
Adicionado: The Italian Job (9654) | Tempo: 39.08s

--- Processando: I Killed My Mother (2009) ---
Enviando requisição...
Buscando detalhes (ID: 26280)...
Adicionado: I Killed My Mother (26280) | Tempo: 39.32s

--- Processando: The Babadook (2014) ---
Enviando requisição...
Buscando detalhes (ID: 242224)...
Adicionado: The Babadook (242224) | Tempo: 39.55s

--- Processando: Gerald's Game (2017) ---
Enviando requisição...
Buscando detalhes (ID: 343674)...
Adicionado: Gerald's Game (343674) | Tempo: 39.79s

--- Processando: As the Gods Will (2014) ---
Enviando requisição...
Buscando detalhes (ID: 241863)...
Adicionado: As the Gods Will (241863) | Tempo: 40.08s

--- Processando: Death Note (2017) ---
Enviando requisição...
Buscando detalhes (ID: 351460)...
Adicionado: Death Note (351460) | Tempo: 40.31s

--- Processando: Up (2009) ---
Enviando requisição...
Buscando detalhes (ID: 14160)...
Adicionado: Up (14160) | Tempo: 40.55s

--- P

Buscando detalhes (ID: 629542)...
Adicionado: The Bad Guys (629542) | Tempo: 51.82s

--- Processando: Fargo (1996) ---
Enviando requisição...
Buscando detalhes (ID: 275)...
Adicionado: Fargo (275) | Tempo: 52.06s

--- Processando: Marley & Me (2008) ---
Enviando requisição...
Buscando detalhes (ID: 14306)...
Adicionado: Marley & Me (14306) | Tempo: 52.29s

--- Processando: Flags of Our Fathers (2006) ---
Enviando requisição...
Buscando detalhes (ID: 3683)...
Adicionado: Flags of Our Fathers (3683) | Tempo: 52.53s

--- Processando: Petite Maman (2021) ---
Enviando requisição...
Buscando detalhes (ID: 749004)...
Adicionado: Petite Maman (749004) | Tempo: 52.77s

--- Processando: Letters from Iwo Jima (2006) ---
Enviando requisição...
Buscando detalhes (ID: 1251)...
Adicionado: Letters from Iwo Jima (1251) | Tempo: 53.00s

--- Processando: The Big Lebowski (1998) ---
Enviando requisição...
Buscando detalhes (ID: 115)...
Adicionado: The Big Lebowski (115) | Tempo: 53.24s

--- Processando: 

...

Buscando detalhes (ID: 29161)...
Adicionado: Phenomena (29161) | Tempo: 126.94s

--- Processando: The Others (2001) ---
Enviando requisição...
Buscando detalhes (ID: 1933)...
Adicionado: The Others (1933) | Tempo: 127.17s

--- Processando: A Page of Madness (1926) ---
Enviando requisição...
Buscando detalhes (ID: 94525)...
Adicionado: A Page of Madness (94525) | Tempo: 127.40s

--- Processando: The Conjuring (2013) ---
Enviando requisição...
Buscando detalhes (ID: 138843)...
Adicionado: The Conjuring (138843) | Tempo: 127.67s

--- Processando: Saw II (2005) ---
Enviando requisição...
Buscando detalhes (ID: 215)...
Adicionado: Saw II (215) | Tempo: 127.90s

--- Processando: The Exorcist (1973) ---
Enviando requisição...
Buscando detalhes (ID: 9552)...
Adicionado: The Exorcist (9552) | Tempo: 128.14s

--- Processando: Central Station (1998) ---
Enviando requisição...
Buscando detalhes (ID: 666)...
Adicionado: Central Station (666) | Tempo: 128.37s

--- Processando: Jennifer's Body (2009)

Enviando requisição...
Buscando detalhes (ID: 849869)...
Adicionado: Kill Boksoon (849869) | Tempo: 140.17s

--- Processando: Harry Potter and the Philosopher's Stone (2001) ---
Enviando requisição...
Buscando detalhes (ID: 671)...
Adicionado: Harry Potter and the Philosopher's Stone (671) | Tempo: 140.40s

--- Processando: Throw Down (2004) ---
Enviando requisição...
Buscando detalhes (ID: 25664)...
Adicionado: Throw Down (25664) | Tempo: 140.64s

--- Processando: La Abuela (The Grandmother) (2021) ---
Enviando requisição...
Buscando detalhes (ID: 661791)...
Adicionado: La Abuela (The Grandmother) (661791) | Tempo: 140.88s

--- Processando: Anatomy of a Fall (2023) ---
Enviando requisição...
Buscando detalhes (ID: 915935)...
Adicionado: Anatomy of a Fall (915935) | Tempo: 141.11s

--- Processando: Bo Burnham: Inside (2021) ---
Enviando requisição...
Buscando detalhes (ID: 823754)...
Adicionado: Bo Burnham: Inside (823754) | Tempo: 141.34s

--- Processando: Police Story (1985) ---
Envi

Adicionado: Cléo from 5 to 7 (499) | Tempo: 152.58s

--- Processando: Coherence (2013) ---
Enviando requisição...
Buscando detalhes (ID: 220289)...
Adicionado: Coherence (220289) | Tempo: 152.81s

--- Processando: Dona Flor and Her Two Husbands (1976) ---
Enviando requisição...
Buscando detalhes (ID: 42234)...
Adicionado: Dona Flor and Her Two Husbands (42234) | Tempo: 153.04s

--- Processando: Z-O-M-B-I-E-S (2018) ---
Enviando requisição...
Buscando detalhes (ID: 483980)...
Adicionado: Z-O-M-B-I-E-S (483980) | Tempo: 153.28s

--- Processando: Battle Royale (2000) ---
Enviando requisição...
Buscando detalhes (ID: 3176)...
Adicionado: Battle Royale (3176) | Tempo: 153.51s

--- Processando: Bring Me the Head of Alfredo Garcia (1974) ---
Enviando requisição...
Buscando detalhes (ID: 11942)...
Adicionado: Bring Me the Head of Alfredo Garcia (11942) | Tempo: 153.83s

--- Processando: Bamboozled (2000) ---
Enviando requisição...
Buscando detalhes (ID: 24664)...
Adicionado: Bamboozled (24664)

Enviando requisição...
Buscando detalhes (ID: 8337)...
Adicionado: They Live (8337) | Tempo: 165.83s

--- Processando: Aliens vs Predator: Requiem (2007) ---
Enviando requisição...
Buscando detalhes (ID: 440)...
Adicionado: Aliens vs Predator: Requiem (440) | Tempo: 166.06s

--- Processando: Hellraiser (1987) ---
Enviando requisição...
Buscando detalhes (ID: 9003)...
Adicionado: Hellraiser (9003) | Tempo: 166.30s

--- Processando: Prometheus (2012) ---
Enviando requisição...
Buscando detalhes (ID: 70981)...
Adicionado: Prometheus (70981) | Tempo: 166.53s

--- Processando: Deep Red (1975) ---
Enviando requisição...
Buscando detalhes (ID: 20126)...
Adicionado: Deep Red (20126) | Tempo: 166.76s

--- Processando: Ilo Ilo (2013) ---
Enviando requisição...
Buscando detalhes (ID: 188598)...
Adicionado: Ilo Ilo (188598) | Tempo: 167.00s

--- Processando: Bulbbul (2020) ---
Enviando requisição...
Buscando detalhes (ID: 714338)...
Adicionado: Bulbbul (714338) | Tempo: 167.23s

--- Processando: I

Buscando detalhes (ID: 294963)...
Adicionado: Bone Tomahawk (294963) | Tempo: 178.50s

--- Processando: The Housemaid (2010) ---
Enviando requisição...
Buscando detalhes (ID: 45202)...
Adicionado: The Housemaid (45202) | Tempo: 178.74s

--- Processando: Anora (2024) ---
Enviando requisição...
Buscando detalhes (ID: 1064213)...
Adicionado: Anora (1064213) | Tempo: 178.97s

--- Processando: Shaolin Soccer (2001) ---
Enviando requisição...
Buscando detalhes (ID: 11770)...
Adicionado: Shaolin Soccer (11770) | Tempo: 179.21s

--- Processando: Peppermint Candy (1999) ---
Enviando requisição...
Buscando detalhes (ID: 41245)...
Adicionado: Peppermint Candy (41245) | Tempo: 179.44s

--- Processando: Bright (2017) ---
Enviando requisição...
Buscando detalhes (ID: 400106)...
Adicionado: Bright (400106) | Tempo: 179.68s

--- Processando: Insomnia (2002) ---
Enviando requisição...
Buscando detalhes (ID: 320)...
Adicionado: Insomnia (320) | Tempo: 179.91s

--- Processando: Her (2013) ---
Enviando re

In [7]:
# Colocando a coluna minhas_notas e ano no dataframe tmdb
for i,filme in my_rating_df.iterrows():
    nome_f = filme['Name']
    nota = filme['Rating']
    ano = filme['Year']
    if nome_f in tmdb_rating_df['title'].values:
        tmdb_rating_df.loc[tmdb_rating_df['title'] == nome_f,'year'] = ano
        tmdb_rating_df.loc[tmdb_rating_df['title'] == nome_f,'minhas_notas'] = nota

In [8]:
# Salvando em um arquivo csv para não precisar refazer a busca
tmdb_rating_df.to_csv('my_movies_tmdb.csv', index=False)

In [9]:
tmdb_rating_df = pd.read_csv('my_movies_tmdb.csv')

In [10]:
tmdb_rating_df.head()

,title,tmdb_id,tmdb_title,year,genre_ids,original_language,popularity,vote_count,vote_average,budget,revenue,runtime,production_company_id,minhas_notas
0,Violet,665901,Violet,2021,[18],en,1.025,25,7.000,0,0,92,20443.0,4.0
1,Hot Fuzz,4638,Hot Fuzz,2007,"[80, 28, 35]",en,5.328,7691,7.562,12000000,80600000,121,443.0,3.5
2,The World's End,107985,The World's End,2013,"[35, 28, 878]",en,4.319,5473,6.807,20000000,46100000,109,443.0,3.0
3,Scott Pilgrim vs. the World,22538,Scott Pilgrim vs. the World,2010,"[28, 35, 10749]",en,8.117,7993,7.491,85000000,51691156,113,2527.0,4.0
4,La La Land,313369,La La Land,2016,"[35, 18, 10749, 10402]",en,12.232,17121,7.900,30000000,509183536,129,491.0,3.0


In [11]:
tmdb_rating_df.isnull().sum() # Verificando se tem valores vazios

title                    0
tmdb_id                  0
tmdb_title               0
year                     0
genre_ids                0
original_language        0
popularity               0
vote_count               0
vote_average             0
budget                   0
revenue                  0
runtime                  0
production_company_id    6
minhas_notas             0
dtype: int64

In [12]:
# Target e prediction
y = tmdb_rating_df.minhas_notas
X = tmdb_rating_df.drop(['minhas_notas'],axis=1)
# Dividindo em treino e validação
X_train_full,X_valid_full,y_train,y_valid = train_test_split(X,y,
                                                                       train_size=0.8,
                                                                       test_size=0.2, 
                                                                       random_state=0)

# Selecionando colunas de interesse
numerical_cols = [col_name for col_name in X_train_full.columns if X_train_full[col_name].dtype in ['int64','float64']]
my_cols = numerical_cols + ['genre_ids','original_language']
my_cols.remove('tmdb_id')
my_cols

['year',
 'popularity',
 'vote_count',
 'vote_average',
 'budget',
 'revenue',
 'runtime',
 'production_company_id',
 'genre_ids',
 'original_language']

In [13]:
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [14]:
# Transformando a variável categórica 'original_language' em numérica
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
label_X_train['original_language'] = ordinal_encoder.fit_transform(X_train[['original_language']])
label_X_valid['original_language'] = ordinal_encoder.transform(X_valid[['original_language']])

In [15]:
# Tirando os generos da lista criando uma linha para cada genero em cada filme
genres_X_train = label_X_train.copy()
genres_X_valid = label_X_valid.copy()

genres_X_train = genres_X_train.explode('genre_ids')
genres_X_valid = genres_X_valid.explode('genre_ids')


In [16]:
# Criando uma coluna para cada gênero e reagrupando os filmes
oh_encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
oh_genres_train = pd.DataFrame(oh_encoder.fit_transform(genres_X_train[['genre_ids']]))
oh_genres_valid = pd.DataFrame(oh_encoder.transform(genres_X_valid[['genre_ids']]))

oh_genres_train.index = genres_X_train.index
oh_genres_valid.index = genres_X_valid.index

genre_drop_train = genres_X_train.drop('genre_ids', axis=1)
genre_drop_valid = genres_X_valid.drop('genre_ids',axis=1)

oh_X_train = pd.concat([genre_drop_train,oh_genres_train], axis=1)
oh_X_valid = pd.concat([genre_drop_valid,oh_genres_valid], axis=1)

oh_X_train_final = oh_X_train.groupby(oh_X_train.index).max()
oh_X_valid_final = oh_X_valid.groupby(oh_X_valid.index).max()

oh_X_train_final.columns = oh_X_train_final.columns.astype(str)
oh_X_valid_final.columns = oh_X_valid_final.columns.astype(str)

In [17]:
# Criando uma função que retorna MAE e predict array
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds),preds


In [18]:
# Verificando o score sem as colunas genre_ids e production_company_id
score,pred = score_dataset(label_X_train.drop(columns=['genre_ids','production_company_id']),
              label_X_valid.drop(columns=['genre_ids','production_company_id']),y_train,y_valid)
score

0.8767592592592595

In [19]:
# Verificando o score sem a coluna production_company_id
score, pred = score_dataset(oh_X_train_final.drop(columns=['production_company_id']),
              oh_X_valid_final.drop(columns=['production_company_id']),y_train,y_valid)
score

1.1229629629629632

In [20]:
new_rating_df = X_valid_full.copy()
new_rating_df['minhas_notas'] = y_valid
new_rating_df['new_rating'] = pred
new_rating_df[['title','new_rating']].sort_values(by='new_rating')

,title,new_rating
626,Triangle of Sadness,1.895
101,Iron Man 2,1.940
452,The Whale,2.065
424,Tangled,2.150
489,Yi Yi,2.160
...,...,...
614,Pixote,3.595
412,Son of the Mask,3.610
656,Primer,3.680
263,"Jeanne Dielman, 23, quai du Commerce, 1080 Bru...",3.680


In [21]:
# Criando uma função que devolve o MAE por modelo
def score_model(model, X_t, X_v, y_t, y_v):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

In [22]:
# Iterando entre modelos de floresta com parâmetros diferentes
for i,model in enumerate(forest_models):
    score = score_model(model,
                oh_X_train_final.drop(columns=['production_company_id']),
                oh_X_valid_final.drop(columns=['production_company_id']),
                y_train,y_valid)
    print(f'model {i} score: {score}')

model 0 score: 1.1145061728395063
model 1 score: 1.1375617283950619
model 2 score: 1.1360802469135804
model 3 score: 1.1129012345679012
model 4 score: 1.1108950617283948


In [23]:
# Criando modelo xgboost
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=500, early_stopping_rounds=5)
xgb_model.fit(oh_X_train_final, y_train,
             eval_set=[(oh_X_valid_final,y_valid)],
             verbose=False)
xgb_predict = xgb_model.predict(oh_X_valid_final)
mean_absolute_error(xgb_predict,y_valid)

1.0723800659179688

In [24]:
new_rating_df['xgb_rating'] = xgb_predict
new_rating_df[['title','minhas_notas','new_rating','xgb_rating']].sort_values(by='xgb_rating').head(10)

,title,minhas_notas,new_rating,xgb_rating
247,The Virgin Suicides,3.5,2.445,2.354639
668,Torment,1.0,2.425,2.354639
251,Day Shift,3.0,2.445,2.354639
249,RRR,4.5,2.490,2.354639
478,Lady Bird,4.0,2.395,2.402213
424,Tangled,2.0,2.150,2.402213
760,Over the Hedge,1.0,2.300,2.402213
626,Triangle of Sadness,1.0,1.895,2.402213
255,Happy Together,4.0,2.405,2.402213
499,Cha Cha Real Smooth,2.5,2.635,2.437286


In [25]:
new_rating_df[['title','minhas_notas','new_rating','xgb_rating']].sort_values(by='xgb_rating').tail(10)

,title,minhas_notas,new_rating,xgb_rating
122,Saw,2.5,3.570,3.124328
204,50/50,3.5,3.050,3.124328
416,Click,2.0,2.935,3.124328
689,Alien,3.0,3.360,3.226242
518,Masculin Féminin,2.0,3.550,3.226242
392,Hancock,1.0,2.820,3.226242
338,Back to the Future Part III,2.5,3.005,3.226242
729,Spider-Man,2.0,3.090,3.226242
145,To All the Boys I've Loved Before,3.0,3.550,3.262286
576,The 400 Blows,2.0,3.285,3.278830
